In [19]:
import numpy as np
import ipynb.fs.defs.triangular_systems as ts

<h1 style="background-color:rgb(181 ,50 ,84);color:white;text-align:center">Test matrices</h1>

In [43]:
A1 = np.array([
    [1, -1, 4],
    [3, 1, 5],
    [1, 3, -1]
])

b1 = np.array([[10], [15], [6]])

A2 = np.array([
    [1, -1, 2, -3],
    [2, 1, 0, -1],
    [0, 2, 1, 1],
    [2, 0, 1, 0]
])

b2 = np.array([[0], [3], [-3], [0]])

n = 5
A3 = np.zeros((n, n))
b3 = np.zeros((n, 1))

for i in range(n):
    for j in range(n):
        A3[i, j] = 1 / (i+j+1)
        
    b3[i] = np.sum(A3[i])

<h1 style="background-color:rgb(181 ,50 ,84);color:white;text-align:center">Gaussian elimination method</h1>

1. First step: definition of the multiplier
$$m_{ik} = \frac{a_{ik}^{(k)}}{a_{kk}^{(k)}},\ \ \ \ i = k+1,\dots,n$$

2. Second step: modification of the complete matrix
$$a_{ij}^{(k+1)} = a_{ij}^{(k)} - m_{ik}a_{kj}^{(k)},\ \ \ \ i = k+1,\dots,n,\ j = k,\dots,n$$
$$ b_i^{(k+1)} = b_i^{(k)} - m_{ik}b_k^{(k)},\ \ \ \ i = k+1,\dots,n$$

In [2]:
def gaussian_elimination(A: np.array, b: np.array) -> np.array:
    """
    Solves a linear system using Gaussian elimination
    
    Parameters:
        A (numpy array): nxn matrix
        b (numpy array): nx1 known vector
        
    Returns:
        x (numpy array): nx1 vector solution of the linear system 
        
    Raises an exception if one of the pivots is vanishing
    """
    
    # Matrix dimensions
    n = A.shape[0]
    
    # Starting linear system
    A_k = A.copy()*1.
    b_k = b.copy()*1.
    
    # Algorithm
    for k in range(n):
        
        # Check if the pivot is zero
        if A_k[k, k] == 0:
            raise Exception("The pivot is zero")
            
        # Definition of the multipliers
        m = A_k[k+1:, k] / A_k[k, k]

        # Modification of the system
        for i in range(k+1, n):
            
            # New matrix A
            for j in range(k, n):
                A_k[i, j] = A_k[i, j] - m[i-k-1] * A_k[k, j]
            
            # New vector b
            b_k[i] = b_k[i] - m[i-k-1] * b_k[k]

    # Solution of the system
    x = ts.backward_substitution(A_k, b_k)
        
    return x

### Testing

In [44]:
# Random linear system
A = A3
b = b3

# Solution of the system
x = gaussian_elimination(A, b)

# Residual vector
r = A @ x - b

# Print results
print(f'x = \n{x}\n')
print(f'r = \n{r}\n')

x = 
[[1.]
 [1.]
 [1.]
 [1.]
 [1.]]

r = 
[[ 0.00000000e+00]
 [ 0.00000000e+00]
 [-2.22044605e-16]
 [ 0.00000000e+00]
 [ 0.00000000e+00]]



<h1 style="background-color:rgb(181 ,50 ,84);color:white;text-align:center">Gaussian elimination for LU factorization</h1>

Using the Gaussian elimination method is equivalent to performing an $LU$ factorization. Defining 

$$m_k = [0\cdots0\ \ m_{k+1,k}\cdots m_{nk}]$$
we can then create the matrices 
$$M_k = I - m_k\hat e_k^T$$ 
so that 
$$L = (M_{n-1}M_{n-2}\cdots M_1),\ \ \ \ \ U = A^{(n)}$$

In [4]:
def gem_LU_factorization(A: np.array) -> [np.array, np.array]:
    """
    Performes an LU factorization using the Gaussian elimination method
    
    Parameters:
        A (numpy array): nxn matrix
        
    Returns:
        L (numpy array): nxn lower triangular matrix with ones on the diagonal
        U (numpy array): nxn upper triangular matrix
        
    Raises an exception if one of the pivots is vanishing
    """
    
    # Matrix dimensions
    n = A.shape[0]
    
    # Starting matrix
    A_k = A.copy()*1.
    
    # Initialization of L
    L = np.eye(n)
    
    # Algorithm
    for k in range(n):
        
        # Check if the pivot is zero
        if A_k[k, k] == 0:
            raise Exception("The pivot is zero")
            
        # Definition of the multipliers
        m = A_k[k+1:, k] / A_k[k, k]

        # Modification of the system
        for i in range(k+1, n):
            
            # New matrix A
            for j in range(k, n):
                A_k[i, j] = A_k[i, j] - m[i-k-1] * A_k[k, j]
            
        # Insert the multipliers into L
        L[k+1:, k] = m
    
    return L, A_k

### Testing

In [45]:
# Random linear system
A = A3
b = b3

# Factorization
L, U = gem_LU_factorization(A)

print(f'A = \n{A}\n')
print(f'L = \n{L}\n')
print(f'U = \n{U}\n')
print(f'LU = \n{L@U}\n')

A = 
[[1.         0.5        0.33333333 0.25       0.2       ]
 [0.5        0.33333333 0.25       0.2        0.16666667]
 [0.33333333 0.25       0.2        0.16666667 0.14285714]
 [0.25       0.2        0.16666667 0.14285714 0.125     ]
 [0.2        0.16666667 0.14285714 0.125      0.11111111]]

L = 
[[1.         0.         0.         0.         0.        ]
 [0.5        1.         0.         0.         0.        ]
 [0.33333333 1.         1.         0.         0.        ]
 [0.25       0.9        1.5        1.         0.        ]
 [0.2        0.8        1.71428571 2.         1.        ]]

U = 
[[ 1.00000000e+00  5.00000000e-01  3.33333333e-01  2.50000000e-01
   2.00000000e-01]
 [ 0.00000000e+00  8.33333333e-02  8.33333333e-02  7.50000000e-02
   6.66666667e-02]
 [ 0.00000000e+00 -1.38777878e-17  5.55555556e-03  8.33333333e-03
   9.52380952e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  3.57142857e-04
   7.14285714e-04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000

In [46]:
# Solution of the linear system
y = ts.forward_substitution(L, b)
x = ts.backward_substitution(U, y)

# Residual vector
r = A @ x - b

# Print results
print(f'x = \n{x}\n')
print(f'r = \n{r}\n')

x = 
[[1.]
 [1.]
 [1.]
 [1.]
 [1.]]

r = 
[[ 4.44089210e-16]
 [ 0.00000000e+00]
 [-2.22044605e-16]
 [ 0.00000000e+00]
 [ 0.00000000e+00]]



<h1 style="background-color:rgb(181 ,50 ,84);color:white;text-align:center">Gaussian elimination with partial pivoting</h1>

In [7]:
def gem_LU_partial_pivoting(A):
    """
    Performes an LU factorization using the Gaussian elimination method and partial pivoting
    Calculates the decomposition of PI@A = L@U
    
    Parameters:
        A (numpy array): nxn matrix
        
    Returns:
        L (numpy array): nxn lower triangular matrix with ones on the diagonal
        U (numpy array): nxn upper triangular matrix
        PI (numpy array): nxn permutation matrix
        
    Raises an exception if one of the pivots is vanishing
    """
    
    # Upper triangular matrix
    U = A.copy()

    # Number of rows of the matrix A
    n = U.shape[0]  

    # Check if the matrix is square
    if U.shape[0] != U.shape[1]:
        raise Exception("The matrix is not square")

    # Algorithm
    L = np.zeros((n,n))
    PI = np.eye(n)

    for k in range(0, n-1):
        
        # Check if the pivot is zero
        if U[k, k] == 0:
            raise Exception("The pivot is zero")
        
        # Partial pivoting
        if U[k, k] < np.max(U[k:, k]):
            r = np.argmax(U[k:, k]) + k
            U[[k, r]] = U[[r, k]]
            L[[k, r]] = L[[r, k]]

            pi = np.eye(n, n)
            pi[[k, r]] = pi[[r, k]]

            PI = np.dot(pi, PI)

        m = U[k+1:, k] / U[k, k]
        z = np.zeros(k+1)
        m = np.concatenate((z, m)).reshape(n, 1)

        e = np.eye(1, n, k)
        M = np.eye(n,n) - np.dot(m, e)

        U = np.dot(M, U)
        L[:, k] += m[:, 0]

    L += np.eye(n)
    
    return L, U, PI

### Testing

In [24]:
# Random linear system
A = A1
b = b1

# Factorization
L, U, PI = gem_LU_partial_pivoting(A)

print(f'A = \n{A}\n')
print(f'L = \n{L}\n')
print(f'U = \n{U}\n')
print(f'PI.T LU = \n{PI.T@L@U}\n')

A = 
[[ 1 -1  4]
 [ 3  1  5]
 [ 1  3 -1]]

L = 
[[ 1.          0.          0.        ]
 [ 0.33333333  1.          0.        ]
 [ 0.33333333 -0.5         1.        ]]

U = 
[[ 3.          1.          5.        ]
 [ 0.          2.66666667 -2.66666667]
 [ 0.          0.          1.        ]]

PI.T LU = 
[[ 1. -1.  4.]
 [ 3.  1.  5.]
 [ 1.  3. -1.]]



In [25]:
# Solution of the linear system
y = ts.forward_substitution(L, PI@b)
x = ts.backward_substitution(U, y)

# Residual vector
r = A @ x - b

# Print results
print(f'x = \n{x}\n')
print(f'r = \n{r}\n')

x = 
[[-6.125]
 [ 5.875]
 [ 5.5  ]]

r = 
[[0.00000000e+00]
 [3.55271368e-15]
 [8.88178420e-16]]



<h1 style="background-color:rgb(181 ,50 ,84);color:white;text-align:center">Gaussian elimination with total pivoting</h1>

In [10]:
def gem_LU_total_pivoting(A):
    """
    Performes an LU factorization using the Gaussian elimination method and partial pivoting
    Calculates the decomposition of PI_L@A@PI_R = L@U
    
    Parameters:
        A (numpy array): nxn matrix
        
    Returns:
        L (numpy array): nxn lower triangular matrix with ones on the diagonal
        U (numpy array): nxn upper triangular matrix
        PI_left (numpy array): nxn left permutation matrix
        PI_right (numpy array): nxn right permutation matrix

    Raises an exception if one of the pivots is vanishing
    """
    
    # Upper triangular matrix
    U = A.copy()
    
    # Number of rows of the matrix A
    n = U.shape[0]  

    # Check if the matrix is square
    if U.shape[0] != U.shape[1]:
        raise Exception("The matrix is not square")

    # Algorithm
    L = np.zeros((n, n))
    PI_left = np.eye(n)
    PI_right = np.eye(n)

    for k in range(0, n-1):

        # Check if the pivot is zero
        if A[k, k] == 0:
            raise Exception("The pivot is zero")

        # Total pivoting
        if U[k, k] < np.max(np.abs(U[k:, k:])):
            r = np.argmax(np.abs(U[k:, k:]))
            r, c = np.unravel_index(r, U[k:, k:].shape)
            r += k
            c += k
            
            U[[k, r]] = U[[r, k]]
            U[:, [k, c]] = U[:, [c, k]]
                        
            L[[k, r]] = L[[r, k]]
            L[:, [k, c]] = L[:, [c, k]]

            pi_left = np.eye(n, n)
            pi_left[[k, r]] = pi_left[[r, k]]
            PI_left = np.dot(pi_left, PI_left)

            pi_right = np.eye(n, n)
            pi_right[[k, c]] = pi_right[[c, k]]
            PI_right = np.dot(PI_right, pi_right)


        m = U[k+1:, k] / U[k, k]
        z = np.zeros(k+1)
        m = np.concatenate((z, m)).reshape(n, 1)

        e = np.eye(1, n, k)
        M = np.eye(n,n) - np.dot(m, e)

        U = np.dot(M, U)
        L[:, k] += m[:, 0]
        
    L += np.eye(n)

    return L, U, PI_left, PI_right

### Testing

In [30]:
# Random linear system
A = A2
b = b2

# Factorization
L, U, PI_left, PI_right = gem_LU_total_pivoting(A)

print(f'A = \n{A}\n')
print(f'L = \n{L}\n')
print(f'U = \n{U}\n')
print(f'LU = \n{L@U}\n')
print(f'PI_left.T LU PI_right.T = \n{PI_left.T @ L @ U @ PI_right.T}\n')

A = 
[[ 1 -1  2 -3]
 [ 2  1  0 -1]
 [ 0  2  1  1]
 [ 2  0  1  0]]

L = 
[[ 1.          0.          0.          0.        ]
 [ 0.          1.          0.          0.        ]
 [-0.33333333  0.16666667  1.          0.        ]
 [ 0.33333333  0.83333333  0.8         1.        ]]

U = 
[[-3.          1.         -1.          2.        ]
 [ 0.          2.          0.          1.        ]
 [ 0.          0.          1.66666667  1.5       ]
 [ 0.          0.          0.         -2.7       ]]

LU = 
[[-3.  1. -1.  2.]
 [ 0.  2.  0.  1.]
 [ 1.  0.  2.  1.]
 [-1.  2.  1.  0.]]

PI_left.T LU PI_right.T = 
[[ 1. -1.  2. -3.]
 [ 2.  1.  0. -1.]
 [ 0.  2.  1.  1.]
 [ 2.  0.  1.  0.]]



In [31]:
# Solution of the linear system
y = ts.forward_substitution(L, PI_left @ b)
z = ts.backward_substitution(U, y)
x = PI_right @ z 

# Residual vector
r = A @ x - b

# Print results
print(f'x = \n{x}\n')
print(f'r = \n{r}\n')

x = 
[[ 1.]
 [ 0.]
 [-2.]
 [-1.]]

r = 
[[ 0.0000000e+00]
 [ 8.8817842e-16]
 [-8.8817842e-16]
 [ 0.0000000e+00]]

